In [16]:
import pandas as pd
from os import walk
import os
import nltk
import pandas as pd
from nltk.probability import FreqDist
from nltk import word_tokenize
from nltk.corpus import stopwords
from utils import * 
import os
import numpy as np
import pandas as pd
import math

In [5]:
sessions = np.arange(25, 76)
data=[]

for session in sessions:
    directory = "./Data/TXT/Session "+str(session)+" - "+str(1945+session)
    for filename in os.listdir(directory):
        f = open(os.path.join(directory, filename), encoding="utf8")
        if filename[0]==".": #ignore hidden files
            continue
        splt = filename.split("_")
        data.append([session, 1945+session, splt[0], f.read()])

        
df_speech = pd.DataFrame(data, columns=['Session','Year','ISO-alpha3 Code','Speech'])

df_speech.tail()

,Session,Year,ISO-alpha3 Code,Speech
8476,75,2020,HRV,"Mr President, Excellencies\nAll protocol obser..."
8477,75,2020,GAB,"Mr. President, Majesties,\nLadies and Gentleme..."
8478,75,2020,MCO,"Mr. President of the General Assembly,\nMr. Se..."
8479,75,2020,AND,"Mr. President,\nMr. Secretary General,\nYour E..."
8480,75,2020,BHR,"In the name of Allah, the most gracious, the m..."


In [6]:
df_speech.to_pickle('./Data/Throughput/Speeches_original.pickle')

In [7]:
preprocessing_steps = ['prep_lower', 
                             'prep_tokenizewords', 
                             'prep_drop_sentenceindicators', 
                             'prep_drop_numbers', 
                             'prep_drop_stopwords',
                       'prep_listtosentence'
                      ]


In [8]:
#df_speech = df_speech.set_index(['Session', 'Year', 'ISO-alpha3 Code'])
df_speech['speech'] = preprocess(df_speech['Speech'], preprocessing_steps)

In [9]:
df_speech.to_pickle('./Data/Throughput/Speeches_preprocessed.pickle')

In [10]:
df_happy = pd.read_excel('./Data/DataForTable2.1.xls')
df_happy = df_happy.rename({'year': 'Year'}, axis=1)
df_mapping = pd.read_csv('./Data/mapping_countries.csv')[['Country name', 'ISO-alpha3 Code']]

In [11]:
pd_final = pd.merge(pd.merge(df_happy, df_mapping, on='Country name'), df_speech, on=['ISO-alpha3 Code', 'Year'])

In [36]:
pd_final['GDP per capita'] = pd_final['Log GDP per capita'].apply(math.exp)

In [58]:
pd_final['GDP per capita_delta_abs'] = pd_final.groupby('Country name')['GDP per capita'].diff()
pd_final['GDP per capita_delta_rel'] = pd_final['GDP per capita_delta_abs'] / pd_final['GDP per capita'].shift(1)
pd_final = pd_final.dropna()
pd_final['sentiment'] = pd.cut(pd_final['GDP per capita_delta_rel'], bins = [-99, 0, 0.03, 10], labels =['negative','neutral', 'positive'])

In [43]:
pd_final[['Country name', 'Year','GDP', 'GDP per capita_delta_abs', 'GDP per capita_delta_rel']].loc[pd_final['Country name'] =='Germany']


,Country name,Year,GDP,GDP per capita_delta_abs,GDP per capita_delta_rel
578,Germany,2007,47100.610273,3151.305529,0.071703
580,Germany,2009,45044.468251,-2598.761642,-0.054546
581,Germany,2010,46999.248284,1954.780033,0.043397
582,Germany,2011,49757.933330,2758.685046,0.058696
583,Germany,2012,49872.426252,114.492922,0.002301
584,Germany,2013,49954.157192,81.730941,0.001639
585,Germany,2014,50845.548282,891.391089,0.017844
586,Germany,2015,51159.273288,313.725006,0.006170
587,Germany,2016,51879.695931,720.422643,0.014082
588,Germany,2017,53071.457080,1191.761149,0.022972


In [60]:
pd_final.head()

,Country name,Year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,...,Session,Speech,speech,GDP,GDP_delta_abs,GDP_delta_rel,GDP per capita,GDP per capita_delta_abs,GDP per capita_delta_rel,sentiment
7,Afghanistan,2015,3.982855,7.634466,0.528597,52.599998,0.388928,0.085082,0.880638,0.491410,...,70,"It gives me great pleasure, on behalf of the I...",gives great pleasure behalf islamic republic a...,2068.265670,-34.118729,-0.016229,2068.265670,-34.118729,-0.016229,negative
8,Afghanistan,2016,4.220169,7.629037,0.559072,52.924999,0.522566,0.047488,0.793246,0.501409,...,71,It is my distinct pleasure to attend the seven...,distinct pleasure attend seventy-first session...,2057.067910,-11.197759,-0.005414,2057.067910,-11.197759,-0.005414,negative
9,Afghanistan,2017,2.661718,7.629684,0.490880,53.250000,0.427011,-0.116068,0.954393,0.435270,...,72,As I stand here before the General Assembly to...,stand general assembly today reminded wise men...,2058.400385,1.332475,0.000648,2058.400385,1.332475,0.000648,neutral
10,Afghanistan,2018,2.694303,7.617663,0.507516,53.575001,0.373536,-0.088125,0.927606,0.384561,...,73,Allow me to start by extending my congratulati...,allow start extending congratulations ms. marí...,2033.804353,-24.596033,-0.011949,2033.804353,-24.596033,-0.011949,negative
11,Afghanistan,2019,2.375092,7.632903,0.419973,53.900002,0.393656,-0.103467,0.923849,0.324108,...,74,It is an honour for me to and stand at this pr...,honour stand prestigious rostrum today represe...,2065.036331,31.231979,0.015356,2065.036331,31.231979,0.015356,neutral


In [61]:
pd_final.to_pickle('./Data/Throughput/Speeches_processed_Happiness.pickle')


In [70]:
pd_final['sentence'] = preprocess(pd_final['Speech'], ['prep_lower'])
pd_final[['sentence', 'sentiment']].to_csv('./Data/Throughput/Speeches_BERT.csv', index=False)

In [68]:
pd

,sentence,sentiment
7,"it gives me great pleasure, on behalf of the i...",negative
8,it is my distinct pleasure to attend the seven...,negative
9,as i stand here before the general assembly to...,neutral
10,allow me to start by extending my congratulati...,negative
11,it is an honour for me to and stand at this pr...,neutral
...,...,...
1692,"let me begin my congratulating you, sir, on yo...",negative
1693,i wish to congratulate mr. miroslav lajčák on ...,positive
1694,it is my honour and pleasure to deliver my mai...,positive
1695,allow me to congratulate mr. tijjani muhammad-...,negative
